In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set to: true - to read results for fibre stiffness model run
#         false - to read results for truss model run
fibre_stiffness = True

#### Define functions
def read_csv_file(fib_stiff,run_num):
    filepath = 'C:\\SEAK Lab\\SEAK Lab Github\\KD3M3\\Truss_AOS\\result\\AOS MOEA Runs\\'

    if fib_stiff:
        filename = 'Fibre Stiffness code run results\\Feas and Stab False\\constraint_adaptive' + str(run_num) + '_hist.csv'
    else: 
        filename = 'Truss code run results\\Feas and Stab False\\constraint_adaptive' + str(run_num) + '_hist.csv' 

    full_filepath = filepath + filename

    with open(full_filepath,newline='') as csvfile:
        data = [row for row in csv.reader(csvfile)]
        iter_num = np.zeros(len(data))
        selected_operators = ["" for x in range(len(data))]
        for i in range(len(data)):
            #print(data[i][0])
            iter_num[i] = int(data[i][0])
            selected_operators[i] = data[i][1]
            
    return iter_num, selected_operators

def create_selection_array(op_sel):
    selection_array = np.zeros(len(op_sel))
    index = 0
    for selection in op_sel:
        if selection == '.RemoveIntersection':
            selection_array[index] = 1
        elif selection == '.ImproveOrientation':
            selection_array[index] = 2
        elif selection == '.AddMember':
            selection_array[index] = 3
        elif selection == '.AddDiagonalMember':
            selection_array[index] = 4
        index += 1
    
    return selection_array

In [ ]:
#### Program Operation
num_runs = 30
selection_dict = {new_list: [] for new_list in range(num_runs)}
for i in range(num_runs):
    print('Reading and creating selection array for run ' + str(i))
    num_iter_i, op_selected_i= read_csv_file(fibre_stiffness, i) 
    selection_dict[i] = create_selection_array(op_selected_i)
    
n_datapoints = len(selection_dict[0])
op1_selected = np.zeros(n_datapoints)
op2_selected = np.zeros(n_datapoints)
op3_selected = np.zeros(n_datapoints)

print('Computing selection frequencies of each operator')
for i in range(n_datapoints):
    for j in range(num_runs):
        current_selection_array = selection_dict[j]
        if current_selection_array[j] == 1:
            op1_selected[i] += 1
        elif current_selection_array[j] == 2:
            op2_selected[i] += 1
        elif current_selection_array[j] == 3:
            op3_selected[i] += 1
            
## Plotting
print('Plotting')
plt.figure()
plt.plot(num_iter_i, op1_selected, 'b*', label='Remove Intersection')
plt.plot(num_iter_i, op2_selected, 'ro', label='Crossover + Mutation')
plt.plot(num_iter_i, op3_selected, 'gv', label='Add Member')
plt.xlabel('numbet of function evaluations')
plt.ylabel('operator seletion frequency over ' + str(num_runs) + ' runs')
plt.legend(loc='upper right')
plt.show()